In [ ]:
pip install Flask transformers PyPDF2 tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.1 MB/s eta 0:00:00


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2024-06-12 13:59:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  59.5MB/s    in 0.2s    

2024-06-12 13:59:16 (59.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
get_ipython().system_raw('./ngrok http 5000 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.10/json/__init__.py", line 293, in load
    return loads(fp.read(),
  File "/usr/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


In [ ]:
# Menyiapkan Flask dan Ngrok
import subprocess
import threading
import json
import time
from flask import Flask, request, render_template_string, jsonify
from transformers import T5Tokenizer, T5ForConditionalGeneration
import PyPDF2
from tqdm import tqdm

app = Flask(__name__)

# Load pre-trained model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def read_pdf(file):
    """
    Read text from a PDF file.
    """
    reader = PyPDF2.PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

def summarize_text(text, update_progress):
    """
    Summarize the input text using T5 model.
    """
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=200, min_length=100, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    update_progress(100)
    return summary

progress = 0

@app.route('/progress')
def get_progress():
    global progress
    return jsonify(progress=progress)

def update_progress(value):
    global progress
    progress = value

@app.route('/', methods=['GET', 'POST'])
def index():
    global progress
    summary = ""
    progress = 0
    if request.method == 'POST':
        file = request.files.get('file')
        text = request.form.get('text')
        if file:
            text = read_pdf(file)
            update_progress(50)  # Update progress to 50% after reading PDF
            time.sleep(1)  # Simulate delay
            summary = summarize_text(text, update_progress)
        elif text:
            update_progress(50)  # Update progress to 50% after receiving text
            time.sleep(1)  # Simulate delay
            summary = summarize_text(text, update_progress)
    return render_template_string('''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Text Summarization</title>
            <style>
                body {
                    font-family: Arial, sans-serif;
                    background-color: #f0f0f0;
                    margin: 0;
                    padding: 0;
                }
                .container {
                    width: 80%;
                    margin: 0 auto;
                    padding: 20px;
                    background-color: #ffffff;
                    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
                    margin-top: 50px;
                    border-radius: 8px;
                }
                h1 {
                    text-align: center;
                }
                form {
                    text-align: center;
                    margin-bottom: 20px;
                }
                input[type="file"],
                textarea {
                    margin: 20px 0;
                }
                textarea {
                    width: 100%;
                    padding: 10px;
                    font-size: 14px;
                    border-radius: 5px;
                    border: 1px solid #ccc;
                    resize: none;
                }
                button {
                    padding: 10px 20px;
                    background-color: #007BFF;
                    color: #ffffff;
                    border: none;
                    border-radius: 5px;
                    cursor: pointer;
                }
                button:hover {
                    background-color: #0056b3;
                }
                .summary {
                    margin-top: 20px;
                    padding: 20px;
                    background-color: #f9f9f9;
                    border-radius: 5px;
                }
                .progress-container {
                    width: 100%;
                    background-color: #e0e0e0;
                    border-radius: 5px;
                    margin-top: 20px;
                }
                .progress-bar {
                    width: 0%;
                    height: 30px;
                    background-color: #76c7c0;
                    border-radius: 5px;
                    text-align: center;
                    line-height: 30px;
                    color: white;
                }
            </style>
            <script>
                function getProgress() {
                    fetch('/progress')
                        .then(response => response.json())
                        .then(data => {
                            document.getElementById('progress-bar').style.width = data.progress + '%';
                            document.getElementById('progress-bar').innerText = data.progress + '%';
                            if (data.progress < 100) {
                                setTimeout(getProgress, 500);
                            }
                        });
                }

                function startProgress() {
                    document.getElementById('progress-bar').style.width = '0%';
                    document.getElementById('progress-bar').innerText = '0%';
                    getProgress();
                }
            </script>
        </head>
        <body>
            <div class="container">
                <h1>Text Summarization</h1>
                <form method="POST" enctype="multipart/form-data" onsubmit="startProgress()">
                    <input type="file" name="file" accept="application/pdf">
                    <textarea name="text" placeholder="Enter text here..." rows="10" cols="50"></textarea>
                    <button type="submit">Summarize</button>
                </form>
                <div class="progress-container">
                    <div id="progress-bar" class="progress-bar">0%</</div>
                </div>
                {% if summary %}
                <div class="summary">
                    <h2>Summary</h2>
                    <p>{{ summary }}</p>
                </div>
                {% endif %}
            </div>
        </body>
        </html>
    ''', summary=summary)

# Menjalankan Flask dan Ngrok di Colab
def run_flask():
    app.run(host='0.0.0.0', port=5000)

flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Menjalankan ngrok
get_ipython().system_raw('./ngrok http 5000 &')
time.sleep(5)  # Beri waktu ngrok untuk memulai
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.10/json/__init__.py", line 293, in load
    return loads(fp.read(),
  File "/usr/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
